In [32]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

In [25]:
#fix seed
seed = 7
numpy.random.seed(seed)

In [13]:
import numpy as np
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)


In [15]:
#load the dataset but only keep the top n words, zero the rest
top_words = 5000
test_split = 0.33
(X_train, y_train), (X_test,y_test) = imdb.load_data(num_words=top_words)

In [16]:
# restore np.load for future normal usage
np.load = np_load_old

In [26]:
#truncate longer reviews and zero-padding shorter reviews
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [43]:
#create model
model = Sequential()
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())

model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
#fit the model
model.fit(X_train, y_train, validation_data=(X_test,y_test), epochs=2, batch_size=128, verbose=1)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 992s 40ms/step - loss: 0.4821 - acc: 0.7306 - val_loss: 0.2964 - val_acc: 0.8738
Epoch 2/2
25000/25000 [==============================] - 949s 38ms/step - loss: 0.2413 - acc: 0.9031 - val_loss: 0.2699 - val_acc: 0.8871


In [45]:
#evaluate the model
scores = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy: %.2f%%" %(scores[1]*100))

25000/25000 [==============================] - 105s 4ms/step
Accuracy: 88.71%
